# Average Directional Index (ADX) and Relative strength index (RSI) strategy creation
In this notebook we are going to create a strategy that uses [Average Directional Index](https://en.wikipedia.org/wiki/Average_directional_movement_index) and [Relative strength index (RSI)](https://en.wikipedia.org/wiki/Relative_strength_index). This notebook will walk you through the following processes:

## What you will learn?
* Downloading all market data.
* Use indicators (SMA, ADX, RSI, Peaks and lows indifications).
* Visualize all market data and metrics
* Create a strategy based on the metrics with the investing algorithm framework
* Backtest the strategy with different configurations and backtest ranges.
* Evaluate the backtest reports to determine the best configuration.

## Why this strategy?
ADX and RSI are enforcing indicators that show the overal direction of a market. 

## Download all required libraries
We are going to donwload the investing algorithm framework and investing algorithm framework indicators plugin.

In [ ]:
print("Checking if investing_algorithm_framework is installed...")

try:
    import investing_algorithm_framework  
    print(f"investing_algorithm_framework is already installed.")
except ImportError:
    print("investing_algorithm_framework is not installed. Installing...")
    import sys
    !{sys.executable} -m pip install investing_algorithm_framework

print("Checking if investing_algorithm_framework indicators plugin is installed...")

try:
    from investing_algorithm_framework import indicators  
    print(f"investing_algorithm_framework_indicators is already installed.")
except ImportError:
    print("investing_algorithm_framework_indicators is not installed. Installing...")
    import sys
    !{sys.executable} -m pip install investing_algorithm_framework[indicators]

Checking if investing_algorithm_framework is installed...


/Users/marcvanduyn/Library/Caches/pypoetry/virtualenvs/investing-algorithm-framework-ygOLr3-a-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


investing_algorithm_framework is already installed.
Checking if investing_algorithm_framework indicators plugin is installed...
investing_algorithm_framework_indicators is not installed. Installing...
zsh:1: no matches found: investing_algorithm_framework[indicators]


## Define data sources
We will use the `ccxt` library to download the data and provide our strategy with the necessary data when backtesting and live trading. We will 
use OHLCV (Open, High, Low, Close and Volume) data from the Bitvavo exchange for the BTC/EUR trading pair.

In [ ]:
import os
from investing_algorithm_framework import CCXTOHLCVMarketDataSource

# Define data storage path. The data provider will write all its downloaded data to this directory. 
# If at this location all files already exist it will just read from the files instead of downloading everything.
storage_path = os.path.join("data")
btc_eur_2h_bitvavo_datasource = CCXTOHLCVMarketDataSource(
    symbol="BTC/EUR",
    market="BITVAVO",
    time_frame="2h",
    storage_path=storage_path
)

## Download and analyse data

In this section we are going to download all required data, deteremine up and down trend, define the backtest date ranges, and visualize the data with our strategy indicators.

We want to indentify up and down trends in the data for optimization of our strategy later during backtesting. For example, if the strategy performs well in up trends but poorly in down trends, we can optimize the strategy to perform better in down trends by changing the strategy configuration. The goal here is to create date ranges for up and down trends so we can evaluate the strategy performance in different market conditions.

In [13]:
from datetime import datetime, timedelta

start_date = datetime(year=2021, month=1, day=1)
end_date = datetime(year=2024, month=6, day=1)
total_date_range = (datetime(year=2022, month=1, day=1), datetime(year=2024, month=6, day=1))

# Our start date for our data is different then our start_date for our backtest range. This is because we will be using indicators such as the 200 sma, 
# which need to have atleast 200 data points before the start date of our backtest range. If we don't do this,
# we can't calculate indicators such as the 200 sma for our strategy.
start_date_data = start_date - timedelta(days=200)

### Download all data for the total date range

In [ ]:
total_data_polars_df = btc_eur_2h_bitvavo_datasource.get_data(
    start_date=total_date_range[0],
    end_date=total_date_range[1],
)

### Define all backtest date ranges and indentify up and down trends. 

We are going to define all the backtesting date ranges that we are going to use to backtest our strategy. The total date range we are going to test is the period of 2021-12-21 - 2024-06-01. For strategy optimization purposes, we are going to determine then up and down trends of this date range and add these to our backtest date ranges. This will result in a date range list that consists of total period (2021-12-21 - 2024-06-01) and the individual up and down trends in this period.


In [ ]:
from investing_algorithm_framework import BacktestDateRange, convert_polars_to_pandas
from investing_algorithm_framework.indicators import detect_up_and_downtrends

backtest_date_ranges = [BacktestDateRange(name="total_date_range", start_date=total_date_range[0], end_date=total_date_range[1])]

# Detect up and down trends 
total_data_pandas_df = convert_polars_to_pandas(total_data_polars_df)
trends = detect_up_and_downtrends(total_data_pandas_df)

for tred_indicator in trends:
    backtest_date_ranges.append(
        BacktestDateRange(name=tred_indicator, start_date=trends[tred_indicator][0], end_date=trends[tred_indicator][1])
    )

### Visualize all the data with strategy indicators

In [ ]:
from datetime import datetime, timedelta
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
from investing_algorithm_framework.indicators import is_uptrend
from investing_algorithm_framework import BacktestDateRange, convert_polars_to_pandas
import tulipy as tp

date_ranges = []

def add_sma(data, period = 50):
    sma_values = tp.sma(data['Close'].to_numpy(), period=period)

    # Pad NaN values for initial rows with a default value, e.g., 0
    sma_values = np.concatenate((np.full(period - 1, 1), sma_values))

    # Assign RSI values to the DataFrame
    data[f"SMA_{period}"] = sma_values
    return data

start_date = datetime(year=2022, month=1, day=1)
start_date_data = start_date - timedelta(days=200)
end_date = datetime(year=2024, month=6, day=1)
df = btc_eur_2h_bitvavo_datasource.get_data(start_date=start_date_data, end_date=end_date, window_size=200)
df = convert_polars_to_pandas(df)

# Calculate the 50-day and 200 day moving average
df = add_sma(df, 50)
df = add_sma(df, 200)

#### Add indicators to our data

#### Define Graphs functions
Here we are going to create functions that will help us create graphs for all our data and metrics. The metrics will be added to the data by our trading bot. We will
create a graph to show the close prices, RSI, ADX, DI+, DI-, bearish divergence between RSI and close price and directional crossover between DI+ and DI-.

In [6]:
from pandas import DataFrame as PandasDataFrame

def create_rsi_graph(data: PandasDataFrame):
    """
    Create a graph for the RSI metric.
    :param data: DataFrame with a 'RSI' column and a Datetime index
    :return: Plotly graph object
    """

    # Check if the index is of type datetime
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("The index of the data should be of type datetime")

    # Check if the 'RSI' column exists
    if 'RSI' not in data.columns:
        raise ValueError("The data should have a 'RSI' column")

    return go.Scatter(
        x=data.index,
        y=data['RSI'],
        mode='lines',
        line=dict(color="green", width=1),
        name="RSI"
    )

def create_prices_graph(data: PandasDataFrame, key="Close"):
    """
    Create a graph for the close prices. By default, the key is set to 'Close'.
    :param data: DataFrame with a 'Close' column and a Datetime index
    :param key: The column to use for the prices
    """

    # Check if the index is of type datetime
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("The index of the data should be of type datetime")

    # Check if the 'Close' column exists
    if key not in data.columns:
        raise ValueError("The data should have a 'Close' column")

    return go.Scatter(
        x=data.index,
        y=data[key],
        mode='lines',
        line=dict(color="blue", width=1),
        name="Close"
    )

def create_adx_graph(data: PandasDataFrame):
    """
    Create a graph for the ADX metric.
    :param data: DataFrame with a 'ADX' column and a Datetime index
    :return: Plotly graph object
    """

    # Check if the index is of type datetime
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("The index of the data should be of type datetime")

    # Check if the 'ADX' column exists
    if 'ADX' not in data.columns:
        raise ValueError("The data should have a 'ADX' column")

    return go.Scatter(
        x=data.index,
        y=data['ADX'],
        mode='lines',
        line=dict(color="green", width=1),
        name="ADX"
    )

def create_di_plus_graph(data: PandasDataFrame):
    """
    Create a graph for the DI+ metric.
    :param data: DataFrame with a '+DI' column and a Datetime index
    :return: Plotly graph object
    """

    # Check if the index is of type datetime
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("The index of the data should be of type datetime")

    # Check if the '+DI' column exists
    if '+DI' not in data.columns:
        raise ValueError("The data should have a '+DI' column")

    return go.Scatter(
        x=data.index,
        y=data['+DI'],
        mode='lines',
        line=dict(color="orange", width=1),
        name="+DI"
    )

def create_di_minus_graph(data: PandasDataFrame):
    """
    Create a graph for the DI- metric.
    :param data: DataFrame with a '-DI' column and a Datetime index
    :return: Plotly graph object
    """

    # Check if the index is of type datetime
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("The index of the data should be of type datetime")

    # Check if the '-DI' column exists
    if '-DI' not in data.columns:
        raise ValueError("The data should have a '-DI' column")

    return go.Scatter(
        x=data.index,
        y=data['-DI'],
        mode='lines',
        line=dict(color="purple", width=1),
        name="-DI"
    )

def create_di_plus_di_minus_crossover_graph(data: PandasDataFrame):
    """
    Create a graph for the DI- and DI+ crossover.
    """

    # Check if the index is of type datetime
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("The index of the data should be of type datetime")

    # Check if the '-DI' and '+DI' columns exist
    if '-DI' not in data.columns or '+DI' not in data.columns:
        raise ValueError("The data should have a '-DI' and '+DI' column")

    # Get all crossover indexes
    crossover_index = data[(data['+DI'] < data['-DI']) & (data['+DI'].shift(1) > data['-DI'].shift(1))].index

    # Use .loc to get the corresponding 'Close' values
    crossover_close_values = data.loc[crossover_index, '+DI']

    return go.Scatter(
        x=crossover_index,
        y=crossover_close_values,
        mode='markers',
        marker=dict(symbol='circle', size=10, color='blue'),
        name='DI- DI+ Crossover'
    )


def create_ema_graph(data: PandasDataFrame, key, color="blue"):
    # Check if the index is of type datetime
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("The index of the data should be of type datetime")

    # Check if the key columns exist
    if key not in data.columns:
        raise ValueError(f"The data should have a {key} column")


    return go.Scatter(
        x=data.index,
        y=data[key],
        mode='lines',
        line=dict(color=color, width=1),
        name=key
    )

def create_crossover_graph(data: PandasDataFrame, key_one, key_two, color="blue"):
    # Check if the index is of type datetime
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("The index of the data should be of type datetime")

    # Check if the key columns exist
    if key_one not in data.columns or key_two not in data.columns:
        raise ValueError(f"The data should have a {key_one} and {key_two} column")

    # Get all crossover indexes
    crossover_index = data[(data[key_one] <= data[key_two]) & (data[key_one].shift(1) >= data[key_two].shift(1))].index

    # Use .loc to get the corresponding 'Close' values
    crossover_close_values = data.loc[crossover_index, key_one]

    return go.Scatter(
        x=crossover_index,
        y=crossover_close_values,
        mode='markers',
        marker=dict(symbol='circle', size=10, color=color),
        name=f'{key_one} {key_two} Crossover'
    )

def create_peaks_chart(data: PandasDataFrame, key="Close", order = 5):

    # Check if the index is of type datetime
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("The index of the data should be of type datetime")

    keys = [f'{key}_highs', f'{key}_lows']

    for key_column in keys:
        if key_column not in data.columns:
            raise ValueError(f"The data should have a '{key_column}' column")

    # Get all peak indexes
    hh_close_index = data[data[f'{key}_highs'] == 1].index
    lh_close_index = data[data[f'{key}_highs'] == -1].index
    ll_close_index = data[data[f'{key}_lows'] == 1].index
    hl_close_index = data[data[f'{key}_lows'] == -1].index

    # Subtract for each index 10 hours
    # hh_close_index = hh_close_index - pd.Timedelta(hours=2 * order)
    # lh_close_index = lh_close_index - pd.Timedelta(hours=2 * order)
    # ll_close_index = ll_close_index - pd.Timedelta(hours=2 * order)
    # hl_close_index = hl_close_index - pd.Timedelta(hours=2 * order)
    # hh_close_index = hh_close_index
    # lh_close_index = lh_close_index
    # ll_close_index = ll_close_index
    # hl_close_index = hl_close_index

    # Use .loc to get the corresponding 'Close' values if the index is in the DataFrame
    hh_close_values = data.loc[hh_close_index, key]
    lh_close_values = data.loc[lh_close_index, key]
    ll_close_values = data.loc[ll_close_index, key]
    hl_close_values = data.loc[hl_close_index, key]

    # Add higher highs
    higher_high_graph = go.Scatter(
        x=hh_close_index,
        # x=dates[hh_close_index - order].values,
        y=hh_close_values,
        mode='markers',
        marker=dict(symbol='triangle-up', size=10, color='blue'),
        name='Higher High Confirmation'
    )

    # Add lower highs
    lower_high_graph = go.Scatter(
        x=lh_close_index,
        y=lh_close_values,
        mode='markers',
        marker=dict(symbol='triangle-down', size=10, color='red'),
        name='Lower High Confirmation'
    )

    # Add lower lows
    lower_lows_graph = go.Scatter(
        x=ll_close_index,
        y=ll_close_values,
        mode='markers',
        marker=dict(symbol='triangle-down', size=10, color='green'),
        name='Lower Lows Confirmation'
    )

    # Add higher lows
    higher_lows = go.Scatter(
        x=hl_close_index,
        y=hl_close_values,
        mode='markers',
        marker=dict(symbol='triangle-up', size=10, color='purple'),
        name='Higher Lows Confirmation'
    )

    return higher_high_graph, lower_high_graph, lower_lows_graph, higher_lows

def create_bullish_divergence_chart(data: PandasDataFrame, key_one, key_two, color = 'red'):
    """
    A bullish divergence occurs when the "<key_one>_lows" makes a new low but the "<key_two>_lows" makes a higher low.

    For example, if the RSI makes a new low but the close price makes a higher low, then we have a bullish divergence.
    """
    divergence_index = data[(data[f'{key_one}_lows'] == -1) & (data[f'{key_two}_lows'] == 1)].index
    divergence_close_values = data.loc[divergence_index, 'Close']

    return go.Scatter(
        x=divergence_index,
        y=divergence_close_values,
        mode='markers',
        marker=dict(symbol='circle', size=10, color=color),
        name='Bullish Divergence'
    )

def create_bearish_divergence_chart(data: PandasDataFrame, key_one, key_two, color = 'red'):
    """
    A bearish divergence occurs when the "<key_one>_highs" makes a new high but the "<key_two>_highs" makes a lower high.

    For example, if the RSI makes a new high but the close price makes a lower high, then we have a bearish divergence.
    """

    # Add divergence charts
    divergence_index = data[(data[f'{key_one}_highs'] == -1) & (data[f'{key_two}_highs'] == 1)].index
    divergence_close_values = data.loc[divergence_index, 'Close']

    return go.Scatter(
        x=divergence_index,
        y=divergence_close_values,
        mode='markers',
        marker=dict(symbol='circle', size=10, color=color),
        name='Bearish Divergence'
    )

def create_entry_graph(data: PandasDataFrame):

    # Iterate over each row in the DataFrame and check if there is a bullish divergence between the RSI and the close price
    # and if there is a crossover between the DI+ and DI- for the last 12 hours (6 candles)
    # Get all crossover indexes
    crossover_index = data[(data['+DI'] <= data['-DI']) & (data['+DI'].shift(1) >= data['-DI'].shift(1))].index
    data['di_crossover'] = 0
    data.loc[crossover_index, 'di_crossover'] = 1

    entry_indexes = []

    for row in data.itertuples():

        if row.di_crossover == 1:
            match = False
            # Check if there was a bullish divergence between the RSI and the close price in the last 2 days
            rsi_window = data.loc[row.Index - pd.Timedelta(days=2):row.Index, 'RSI_lows']
            close_window = data.loc[row.Index - pd.Timedelta(days=2):row.Index, 'Close_lows']

            # Go over each row and check if there is a bullish divergence between the RSI and the close price
            for rsi_row, close_row in zip(rsi_window, close_window):

                if rsi_row == -1 and close_row == 1:
                    entry_indexes.append(row.Index)
                    match = True
                    break

            if not match:
                # Check if the RSI had decreased
                rsi_window = data.loc[row.Index - pd.Timedelta(days=1):row.Index, 'RSI']
                rsi_diff = rsi_window.diff().mean()

                if rsi_diff < -2:
                    entry_indexes.append(row.Index)

    entry_close_values = data.loc[entry_indexes, 'Close']
    return go.Scatter(
        x=entry_indexes,
        y=entry_close_values,
        mode='markers',
        marker=dict(symbol='circle', size=10, color='green'),
        name='Entry Signal'
    )

In [9]:
from investing_algorithm_framework import BacktestDateRange

current_date_range = BacktestDateRange(
    start_date=datetime(2023, 12, 1),
    end_date=datetime(2024, 6, 1),
    name="current"
)
up_turn_date_range = BacktestDateRange(
    start_date=datetime(2022, 12, 20),
    end_date=datetime(2023, 6, 1),
    name="up_turn"
)
sideways_date_range = BacktestDateRange(
    start_date=datetime(2022, 6, 10),
    end_date=datetime(2023, 1, 10),
    name="sideways"
)
down_turn_date_range = BacktestDateRange(
    start_date=datetime(2021, 12, 21),
    end_date=datetime(2022, 6, 20),
    name="down_turn"
)

date_ranges = [current_date_range, up_turn_date_range, sideways_date_range, down_turn_date_range]

## Download all market data and show the metrics and entry points

In [12]:
import pandas as pd
from plotly.subplots import make_subplots
from plotly import graph_objects as go
from datetime import datetime
from investing_algorithm_framework import convert_pandas_to_polars
from challenger_three import TestStrategy


for date_range in date_ranges:
    start_date = date_range.start_date
    end_date = date_range.end_date
    polars_df = data_source.get_data(start_date=start_date, window_size=200)

    # Convert to pandas df for easy usage with plotly
    df = convert_pandas_to_polars(polars_df)

    # Add all metrics to the pandas df with the use of the functions in the trading strategy
    df = TestStrategy.add_rsi(df, period=14)
    df = TestStrategy.add_adx(df, period=14)
    df = TestStrategy.add_peaks(df, key="RSI", order=3, k=2)
    df = TestStrategy.add_peaks(df, key="Close", order=3, k=2)
    df = TestStrategy.add_ema(data=df, key="Close", period=50)
    df = TestStrategy.add_ema(data=df, key="Close", period=200)

    # Create all graphs
    fig = make_subplots(rows=7, cols=1, shared_xaxes=True, vertical_spacing=0.02)
    fig.add_trace(create_prices_graph(df), row=1, col=1)
    fig.add_trace(create_rsi_graph(df), row=2, col=1)
    fig.add_trace(create_adx_graph(df), row=3, col=1)
    fig.add_trace(create_di_plus_graph(df), row=3, col=1)
    fig.add_trace(create_di_minus_graph(df), row=3, col=1)
    fig.add_trace(create_di_plus_di_minus_crossover_graph(df), row=3, col=1)
    fig.add_trace(create_prices_graph(df), row=4, col=1)
    fig.add_trace(create_ema_graph(df, key="EMA_50", color="blue"), row=7, col=1)
    fig.add_trace(create_ema_graph(df, key="EMA_200", color="red"), row=7, col=1)
    fig.add_trace(create_crossover_graph(df, key_one="EMA_50", key_two="EMA_200", color="green"), row=7, col=1)
    close_higher_high_graph, close_lower_high_graph, close_lower_lows_graph, close_higher_lows = \
        create_peaks_chart(df, key="Close")
    rsi_higher_high_graph, rsi_lower_high_graph, rsi_lower_lows_graph, rsi_higher_lows = \
        create_peaks_chart(df, key="RSI")
    bullish_divergence_chart = create_bullish_divergence_chart(df, key_one="RSI", key_two="Close", color='green')
    bearish_divergence_chart = create_bearish_divergence_chart(df, key_one="RSI", key_two="Close", color='red')
    fig.add_trace(close_higher_high_graph, row=4, col=1)
    fig.add_trace(close_lower_high_graph, row=4, col=1)
    fig.add_trace(close_lower_lows_graph, row=4, col=1)
    fig.add_trace(close_higher_lows, row=4, col=1)
    fig.add_trace(rsi_higher_high_graph, row=5, col=1)
    fig.add_trace(rsi_lower_high_graph, row=5, col=1)
    fig.add_trace(rsi_lower_lows_graph, row=5, col=1)
    fig.add_trace(rsi_higher_lows, row=5, col=1)
    fig.add_trace(create_rsi_graph(df), row=5, col=1)
    fig.add_trace(create_entry_graph(df), row=1, col=1)
    fig.add_trace(create_prices_graph(df), row=6, col=1)
    fig.add_trace(bullish_divergence_chart, row=6, col=1)
    fig.add_trace(bearish_divergence_chart, row=6, col=1)
    fig.update_layout(
        title={'text': f'Date range: "{date_range.name}" Metrics and entry points', 'x':0.5},
        autosize=True,
        height=1000
    )
    fig.show()

## Run backtests

In [11]:
from datetime import datetime
from investing_algorithm_framework import create_app, RESOURCE_DIRECTORY, SYMBOLS, PortfolioConfiguration, BacktestDateRange, BacktestReportsEvaluation, pretty_print_backtest_reports_evaluation
from challenger_three import create_algorithm as create_algorithm_three
# from challenger_three_alternative import create_algorithm as create_algorithm_alternative
from alternative import create_algorithm as create_algorithm_alternative

app = create_app(config={RESOURCE_DIRECTORY: "example_strategies_dump", SYMBOLS: ["BTC/EUR"]})

# Create a portfolio configuration with an initial balance of 1000 EUR for backtesting
app.add_portfolio_configuration(PortfolioConfiguration(market="BITVAVO", initial_balance=1000, trading_symbol="EUR"))

# Import all the algorithms with different parameters and register them for backtesting
primary_v1 = create_algorithm_three(name="challenge_three_v1", peaks_order=2, peaks_k=2)
primary_v2 = create_algorithm_three(name="challenge_three_v2", peaks_order=4, peaks_k=2)
alternative_v1 = create_algorithm_alternative(name="alternative_v1", peaks_order=2, peaks_k=2)
alternative_v2 = create_algorithm_alternative(name="alternative_v2", peaks_order=4, peaks_k=2)
algorithms = [primary_v1, primary_v2, alternative_v1, alternative_v2]

# Run the backtests for the different algorithms and date ranges
reports = app.run_backtests(algorithms=algorithms, date_ranges=date_ranges)

# Perform an evaluation of the backtest reports to see which algorithm performed the best
evaluation = BacktestReportsEvaluation(reports)

# Print the evaluation
pretty_print_backtest_reports_evaluation(evaluation)

Running backtests for date range: current 2023-12-01 00:00:00 - 2024-06-01 00:00:00 for a total of 4 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:03<00:00,  1.95s/it]
Running backtest for algorithm with name challenge_three_v1: 100%|██████████| 2197/2197 [00:37<00:00, 59.07it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 107.82it/s]
Running backtest for algorithm with name challenge_three_v2: 100%|██████████| 2197/2197 [00:35<00:00, 61.50it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 29.05it/s]
Running backtest for algorithm with name alternative_v1: 100%|██████████| 2197/2197 [00:36<00:00, 59.69it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 151.84it/s]
Running backtest for algorithm with name alternative_v2: 100%|██████████| 2197/2197 [00:35<00:00, 61.91it/s]


Running backtests for date range: up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00 for a total of 4 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 89.00it/s]
Running backtest for algorithm with name challenge_three_v1: 100%|██████████| 1957/1957 [00:36<00:00, 54.32it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 222.23it/s]
Running backtest for algorithm with name challenge_three_v2: 100%|██████████| 1957/1957 [00:36<00:00, 53.63it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 138.66it/s]
Running backtest for algorithm with name alternative_v1: 100%|██████████| 1957/1957 [00:33<00:00, 57.94it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 177.16it/s]
Running backtest for algorithm with name alternative_v2: 100%|██████████| 1957/1957 [00:31<00:00, 62.47it/s]


Running backtests for date range: sideways 2022-06-10 00:00:00 - 2023-01-10 00:00:00 for a total of 4 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 163.77it/s]
Running backtest for algorithm with name challenge_three_v1: 100%|██████████| 2569/2569 [00:41<00:00, 61.72it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 81.91it/s]
Running backtest for algorithm with name challenge_three_v2: 100%|██████████| 2569/2569 [00:44<00:00, 57.38it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 136.57it/s]
Running backtest for algorithm with name alternative_v1: 100%|██████████| 2569/2569 [00:43<00:00, 59.55it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 187.67it/s]
Running backtest for algorithm with name alternative_v2: 100%|██████████| 2569/2569 [00:41<00:00, 61.77it/s]


Running backtests for date range: down_turn 2021-12-21 00:00:00 - 2022-06-20 00:00:00 for a total of 4 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 60.88it/s]
Running backtest for algorithm with name challenge_three_v1: 100%|██████████| 2173/2173 [00:35<00:00, 61.51it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 172.52it/s]
Running backtest for algorithm with name challenge_three_v2: 100%|██████████| 2173/2173 [00:37<00:00, 58.32it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 24.31it/s]
Running backtest for algorithm with name alternative_v1: 100%|██████████| 2173/2173 [00:39<00:00, 54.42it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 299.54it/s]
Running backtest for algorithm with name alternative_v2: 100%|██████████| 2173/2173 [00:37<00:00, 58.25it/s]



              :%%%#+-          .=*#%%%      Backtest reports evaluation
              *%%%%%%%+------=*%%%%%%%-     ---------------------------
              *%%%%%%%%%%%%%%%%%%%%%%%-     Number of reports: 16 backtest reports
              .%%%%%%%%%%%%%%%%%%%%%%#      Largest overall profit: (Algorithm alternative_v1) 131.5504 EUR 13.1550% (up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00) 
               #%%%####%%%%%%%%**#%%%+      Largest overall growth: (Algorithm alternative_v1) 138.6029 EUR 13.8603% (up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00)
         .:-+*%%%%- -+..#%%%+.+-  +%%%#*=-: 
          .:-=*%%%%. += .%%#  -+.-%%%%=-:.. 
          .:=+#%%%%%*###%%%%#*+#%%%%%%*+-:  
                +%%%%%%%%%%%%%%%%%%%=       
            :++  .=#%%%%%%%%%%%%%*-         
           :++:      :+%%%%%%#-.            
          :++:        .%%%%%#=              
         :++:        .#%%%%%#*=             
        :++-        :%%%%%%%%%+=            
       .++-        -%%%%%%%%

# Run winning algorithm backtest

In [13]:
from datetime import datetime

from investing_algorithm_framework import create_app, RESOURCE_DIRECTORY, SYMBOLS, PortfolioConfiguration, BacktestDateRange

# Get the best performing algorithm based on growth for the down turn date range

# Get the best performing algorithm based on growth for the up turn date range

# Get the best performing algorithm based on growth for the sideways date range

# Get the best performing algorithm based on growth for the current date range

from challenger_three import create_algorithm as create_algorithm_three


app = create_app(
    config={
        RESOURCE_DIRECTORY: "example_strategies_dump",
        SYMBOLS: ["BTC/EUR"]
    }
)
app.add_portfolio_configuration(
    PortfolioConfiguration(
        market="BITVAVO",
        initial_balance=1000,
        trading_symbol="EUR",
    )
)
up_turn_date_range = BacktestDateRange(
    start_date=datetime(2022, 12, 20),
    end_date=datetime(2023, 6, 1),
    name="up_turn"
)
sideways_date_range = BacktestDateRange(
    start_date=datetime(2022, 6, 10),
    end_date=datetime(2023, 1, 10),
    name="sideways"
)
down_turn_date_range = BacktestDateRange(
    start_date=datetime(2021, 12, 21),
    end_date=datetime(2022, 6, 20),
    name="down_turn"
)
date_range = BacktestDateRange(
    start_date=datetime(2024, 1, 21),
    end_date=datetime(2024, 2, 20),
    name="down_turn"
)
challenge_three = create_algorithm_three(name="challenge_three")
report = app.run_backtest(algorithm=challenge_three, backtest_date_range=date_range)

Preparing backtest market data: 100%|██████████| 2/2 [00:02<00:00,  1.17s/it]
Running backtest for algorithm with name challenge_three: 100%|██████████| 361/361 [00:05<00:00, 60.47it/s]


In [14]:
from investing_algorithm_framework import pretty_print_backtest

pretty_print_backtest(report)


                  :%%%#+-          .=*#%%%        Backtest report
                  *%%%%%%%+------=*%%%%%%%-       ---------------------------
                  *%%%%%%%%%%%%%%%%%%%%%%%-       Start date: 2024-01-21 00:00:00
                  .%%%%%%%%%%%%%%%%%%%%%%#        End date: 2024-02-20 00:00:00
                   #%%%####%%%%%%%%**#%%%+        Number of days: 30 
             .:-+*%%%%- -+..#%%%+.+-  +%%%#*=-:   Number of runs: 361
              .:-=*%%%%. += .%%#  -+.-%%%%=-:..   Number of orders: 6
              .:=+#%%%%%*###%%%%#*+#%%%%%%*+-:    Initial balance: 1000.0
                    +%%%%%%%%%%%%%%%%%%%=         Final balance: 1043.9684
                :++  .=#%%%%%%%%%%%%%*-           Total net gain: 43.9684 4.397%
               :++:      :+%%%%%%#-.              Growth: 43.9684 4.397%
              :++:        .%%%%%#=                Number of trades closed: 3
             :++:        .#%%%%%#*=               Number of trades open(end of backtest): 0
           

# Deploy the winning strategy

In [2]:
import os
from investing_algorithm_framework.deployment import deploy_to_azure_functions

# Create temporary deployment directory
deployment_directory = "deployment"
os.makedirs(deployment_directory, exist_ok=True)

# Copy the strategy files to the deployment directory

